In [66]:
from bs4 import BeautifulSoup
from urllib.parse import urlparse
import re
import requests
import html2text
import validators
import itertools

policies_url = "https://adminguide.stanford.edu"

def uri_validator(x):
    try:
        result = urlparse(x)
        return result.scheme is not None
    except:
        return False

def process_policy_document(policy):
    policy_text = str(policy)
    s = BeautifulSoup(policy_text)
    content = s.find("div",id="content")
    title = content.find("h1").text
    text = html2text.html2text(str(content))
    return (title, text)

def parse_section_links(chapter_url):
    policy_page = requests.get(policy_section)
    soup = BeautifulSoup(policy_page.text)
    body = soup.findAll("div", id=re.compile("my-policies-*"))
    anchors = [child.findChildren("a", href=True) for child in body]
    relative_links = [anchor["href"] for anchor in list(itertools.chain(*anchors))]
    links = [f"{policies_url}{link}" for link in relative_links]
    return links
    
    
page = requests.get(policies_url)
soup = BeautifulSoup(page.text)

# Scrape policy links from webpage
policy_toc = soup.find("div", id="block-bean-chapters")
chaptersList = policy_toc.find("ol")
policy_link_tags = policy_toc.findChildren("a",href=True)
policy_links = [el["href"] for el in policy_link_tags]

policy_list = []
for policy_section in policy_links:
    chapter_links = parse_section_links(policy_section)    
    for link in chapter_links:
        doc = requests.get(link)
        policy_list.append(process_policy_document(doc.text))

print(f"Scraped {len(policy_list)} policy from {princeton_hr_policies_url}.")

Scraped 166 policy from https://adminguide.stanford.edu.


In [71]:
for policy in policy_list:
    if policy is not None:
        title = policy[0].replace(" ", "_")
        contents = policy[1]
        f = open(f"/Users/erik.beerepoot/Documents/policies/stanford/{title}.md","w+")
        f.write(contents)
        f.close()

FileNotFoundError: [Errno 2] No such file or directory: '/Users/erik.beerepoot/Documents/policies/stanford/_12.2.15_Global_Trial/Probation_Periods_.md'